In [ ]:
import sys, os, csv, argparse
import pandas as pd
sys.path.append('..')
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from utility import *

"""
TODO: Execture
"""
if __name__ == '__main__':
    num_data = 2000
    MAX_Iter = 1000 #20 itr/epoch
    BATCH_SIZE = 100
    
    num_func = 1
    num_model = 0
    
    exec('f = f%s' % num_func)
    exec('net = Net%s().cuda().double() ' % num_model)
    
    weight_save_file_name = 'func' + str(num_func) + '_model' + str(num_model) + '_weight.dat'
    
    loader = UtiData.DataLoader(dataset=make_feature(num_data, f), 
                                batch_size=BATCH_SIZE, 
                                shuffle=True, num_workers=1)
    
    #criterion = nn.MSELoss().cuda()
    criterion = nn.MSELoss().cuda()
    
    #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.005)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=0.0000)
    
    loss_total = []
    all_params = np.array([])
    for iter in range(0,MAX_Iter):
        running_loss = 0.0
        optimizer.zero_grad()
        for step_i, (x_batch, y_batch) in enumerate(loader):
            if x_batch.size(0) != BATCH_SIZE or y_batch.size(0) != BATCH_SIZE:
                continue
            
            x_input = Variable(x_batch.type(torch.DoubleTensor).cuda())
            y_input = Variable(y_batch.type(torch.DoubleTensor).cuda())
            
            y_hat = net(x_input)
            
            loss = criterion(y_hat, y_input)
            loss.backward()
        
            # Does the update
            optimizer.step()
            running_loss += loss.data[0]
            
        loss_total.append(running_loss/(num_data/BATCH_SIZE))
        print('Iteration %3d: loss %.10f' % (iter, running_loss/(num_data/BATCH_SIZE)))
        
        if not iter % (3*num_data//BATCH_SIZE):
            epoch_params = np.array([])
            for one_name, one_param in net.named_parameters():
                if 'weight' in one_name:
                    epoch_params = np.append(epoch_params, one_param.cpu().data.numpy())
            if not all_params.size:
                all_params = epoch_params
            else:
                all_params = np.vstack((all_params, epoch_params))
            weight_df = pd.DataFrame(np.transpose(all_params))
            weight_df.to_csv(weight_save_file_name, index=False)

    x_show = np.linspace(0,1,num_data) 
    y_show = f(x_show)
    yhat_test = net(Variable(torch.from_numpy(x_show).unsqueeze(1).type(torch.DoubleTensor).cuda())).data.cpu().numpy().squeeze(1)
    
    yhat_show = []
    for i in range(len(yhat_test)):
        yhat_show.append(yhat_test[i])
    
    plt.figure(1), 
    plt.plot(x_show, y_show, 'b--'), 
    plt.plot(x_show, yhat_test, 'g'), plt.grid(True)

    plt.figure(2), 
    plt.semilogy(loss_total, 'r')
    plt.xlabel('Number of iterations')
    plt.ylabel('MSE'), plt.grid(True)
    
    print('Finish running the code!')

In [ ]:
color = 'rgb'
num_func = 1
num_model = 0
weight_save_file_name = 'func' + str(num_func) + '_model' + str(num_model) + '_weight.dat'
weight_read = pd.read_csv(weight_save_file_name, header=None)
print(weight_read[0])
#plt.figure(1), 
#plt.plot(data_read["x"], data_read["yhat"], color[num_model], label='Model'+str(num_model)), plt.grid(True)
